# 다중 로지스틱 회귀 모델(multinomial OR softmax)

## 비용함수: 이진 로지스틱 회귀처럼 교차 엔트로피 활용
### sigmoid -> softmax로 확장
- logit <-> softmax 역함수 관계

### LogisticRegression
- penalty: 모델의 복잡도 조절
- C: penalty 정도
- solver: 다양한 해결 방법을 고를 수 있음
- max_iter: 반복 횟수
- multi_class
    1. 'ovr': 이진 로지스틱 회귀를 모든 라벨에 대해서 적용
    2. 'multinomial': softmax 로지스틱 회귀
    3. 'auto': 데이터에 따라 자동으로 적용

## 다중 로지스틱 회귀 문제

### breast cancer dataset

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10, max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [9]:
y_train_pred = clf.predict(X_train_scaled)
y_pred = clf.predict(X_test_scaled)

print(f'train 정확도:{(y_train_pred == y_train).mean() * 100: .2f}%')
print(f'test 정확도:{(y_pred == y_test).mean() * 100: .2f}%')

train 정확도: 99.74%
test 정확도: 96.28%


### tips dataset

In [10]:
import pandas as pd
tips = pd.read_csv("tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


#### tips를 4분위로 이산화, 나머지 feature를 input으로 분류하는 모델 학습

In [11]:
tips.tip = pd.cut(tips['tip'], bins=3, labels=['low', 'middle', 'high'])
tips['smoker'] = tips['smoker'].map({'No': 0, 'Yes': 1})
tips['day'] = tips['day'].map({'Thur':0, "Fri":1, "Sat":2, "Sun":3})
tips['time'] = tips['time'].map({'Lunch': 0, 'Dinner': 1})

In [12]:
X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns='tip'), tips['tip'], test_size=0.33, random_state=1234)

In [13]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
clf = LogisticRegression(penalty='l2', solver='sag', random_state=1234, C=10, max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)
y_train_pred = clf.predict(X_train_scaled)
y_pred = clf.predict(X_test_scaled)

print(f'train 정확도:{(y_train_pred == y_train).mean() * 100: .2f}%')
print(f'test 정확도:{(y_pred == y_test).mean() * 100: .2f}%')

train 정확도: 88.96%
test 정확도: 80.25%


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## 회귀에서 성능 지표

- 회귀의 성능 지표
    1. MAE(Mean Absolute Error): 평균 절댓값 오차
    2. RMSE(Root Mean Squared Error): 루트 평균 제곱 오차
    3. R2 score: 분류기의 score method로 구할 수 있음

In [18]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import SGDRegressor

X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)


reg = SGDRegressor(penalty=None, max_iter=10000, learning_rate='constant', eta0=0.1)
reg = reg.fit(X_train, y_train)
y_pred_train = reg.predict(X_train)
y_pred = reg.predict(X_test)
print('테스트 데이터셋 MAE:', np.abs(y_pred - y_test).mean())
print('테스트 데이터셋 MSE:', ((y_pred - y_test) ** 2).mean())
print('테스트 데이터셋 RMSE:', np.sqrt(((y_pred - y_test) ** 2).mean()))
print('테스트 데이터셋 R2 score', reg.score(X_test, y_test))

테스트 데이터셋 MAE: 43.28230374282563
테스트 데이터셋 MSE: 2844.2114311754995
테스트 데이터셋 RMSE: 53.331148789197286
테스트 데이터셋 R2 score 0.5046112383727555


## 분류에서 성능 지표
- 분류의 성능 지표
    1. 정확도(accuracy): 전체 테스트 샘플 중 정확히 분류한 비율
    2. 정밀도(precision): 1이라고 예측한 샘플 중에서 실제 1인 비율(정밀도가 높을 경우 재현율은 낮음)
    3. 재현율(recall): 실제 1인 샘플 중에서 분류기가 1이라고 예측한 비율(재현율이 높을 경우 정밀도는 낮음)
    4. F1-score: 재현율과 정밀도의 조화 평균(2 * ((recall * precision) / (recall + precision)))

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
clf = LogisticRegression(random_state=1234, solver='sag', penalty='l2', C=10, max_iter=10000, multi_class='multinomial')
clf.fit(X_train_scaled, y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=10, max_iter=10000, multi_class='multinomial',
                   random_state=1234, solver='sag')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
y_pred = clf.predict(X_test_scaled)
print(f'정확도:{(y_pred == y_test).mean() : }')
print(f'정확도:{clf.score(X_test_scaled,y_test): }')
print(f'정확도:{accuracy_score(y_test,y_pred): }')  # y_test가 먼저, y_pred가 그 다음

정확도: 0.9627659574468085
정확도: 0.9627659574468085
정확도: 0.9627659574468085


In [ ]:
print(f'정밀도:{precision_score(y_test,y_pred): }')  # y_test가 먼저, y_pred가 그 다음

정밀도: 0.9508196721311475


In [ ]:
print(f'재현율:{recall_score(y_test,y_pred): }')  # y_test가 먼저, y_pred가 그 다음

재현율: 0.9914529914529915


In [ ]:
print(f'F1 score:{f1_score(y_test,y_pred): }')  # y_test가 먼저, y_pred가 그 다음

F1 score: 0.9707112970711297


In [6]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        71
           1       0.95      0.99      0.97       117

    accuracy                           0.96       188
   macro avg       0.97      0.95      0.96       188
weighted avg       0.96      0.96      0.96       188

